In [1]:
!pip install -q -U transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q -U bitsandbytes einops sentencepiece

In [2]:
from peft import PeftModel

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 6.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu'), PosixPath('/usr/local/cuda/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


In [3]:
import torch

In [4]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, BitsAndBytesConfig

In [5]:
model_id = "decapoda-research/llama-7b-hf"

In [6]:
bnbConfig = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

In [8]:
model = LlamaForCausalLM.from_pretrained(model_id,
                                         load_in_4bit=True,
                                         torch_dtype=torch.bfloat16, #float16,
                                         quantization_config=bnbConfig,
                                         device_map="auto")

Loading checkpoint shards:   0%|          | 0/33 [00:00<?, ?it/s]

In [10]:
model = PeftModel.from_pretrained(model, "SKumari/Llama_train_sk")

In [13]:
tokenizer = LlamaTokenizer.from_pretrained(model_id)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LLaMATokenizer'. 
The class this function is called from is 'LlamaTokenizer'.


In [20]:
custom_prompt = "I want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old."

PROMPT =f'''Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.


### Instruction:
{custom_prompt}
### Response:
'''

In [21]:
%%time

inputs = tokenizer(
    PROMPT,
    return_tensors="pt",
)
input_ids = inputs["input_ids"].cuda()

CPU times: user 2.53 ms, sys: 958 µs, total: 3.49 ms
Wall time: 2.7 ms


In [22]:
generation_config = GenerationConfig(
    temperature=0.01,
    top_p=0.9,
    typical_p=0.9,
    repetition_penalty=2.0,
    encoder_repetition_penalty=2.0,
    top_k=40,
    renormalize_logits=True,
    do_sample=True,
    num_beams=2,
    num_return_sequences=1,
    remove_invalid_values=True
)
print("Generating...")
generation_output = model.generate(
    input_ids=input_ids,
    generation_config=generation_config,
    return_dict_in_generate=True,
    output_scores=False,
    max_new_tokens=750,
)


for s in generation_output.sequences:
    print(tokenizer.decode(s))

Generating...
 ⁇  Below is an instruction that describes a task. Write a response in detail that appropriately completes the request.


### Instruction:
I want to invest in US Treasury Securities. I am looking for long term investments but I want minimal risk. I prefer stable income because I am already 42 years old.
### Response:
You can invest in US Treasury Securities by purchasing Treasury bonds, notes, and bills. These securities are issued by the US government and provide a safe and reliable investment option. Treasury bonds and notes typically have longer maturities, ranging from 10 to 30 years. Treasury bills, on the other hand, have shorter maturities, typically ranging from 1 to 10 years. Treasury bonds and notes offer higher yields compared to Treasury bills. The choice between long term and short term investments will depend on your investment goals and risk tolerance.

It's important to note that while US Treasury Securities are considered low-risk investments, they are no